In [1]:
# import tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from prepare2 import *

# remove max column limit
pd.set_option('display.max_columns', None)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%%time

# load main match frame
mf = prepare_atp()
mf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50470 entries, 1999-01-04 to 2019-11-24
Data columns (total 83 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   tourney_id                        50470 non-null  object 
 1   tourney_name                      50470 non-null  object 
 2   surface                           50470 non-null  object 
 3   draw_size                         50470 non-null  int64  
 4   tourney_level                     50470 non-null  object 
 5   match_num                         50470 non-null  int64  
 6   score                             50470 non-null  object 
 7   best_of                           50470 non-null  int64  
 8   round                             50470 non-null  object 
 9   minutes                           50470 non-null  float64
 10  player_1                          50470 non-null  object 
 11  player_2                          50470 non-null  

In [4]:
%%time

# read workable frame from workable .csv file
wf = pd.read_csv('takes_too_long.csv', index_col = 0)
wf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50470 entries, 0 to 50469
Data columns (total 84 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   tourney_id                        50470 non-null  object 
 1   tourney_name                      50470 non-null  object 
 2   surface                           50470 non-null  object 
 3   draw_size                         50470 non-null  int64  
 4   tourney_level                     50470 non-null  object 
 5   tourney_date                      50470 non-null  object 
 6   match_num                         50470 non-null  int64  
 7   score                             50470 non-null  object 
 8   best_of                           50470 non-null  int64  
 9   round                             50470 non-null  object 
 10  minutes                           50470 non-null  float64
 11  player_1                          50470 non-null  object 
 12  play

#### Some planning, in case you get lost in code, Mason.
- rename columns
- come up with rolling aggregate for surface win

In [5]:
# clean up player columns and review
wf.columns = wf.columns.str.replace('player_1', 'player1').str.replace('player_2', 'player2')
wf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50470 entries, 0 to 50469
Data columns (total 84 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   tourney_id                       50470 non-null  object 
 1   tourney_name                     50470 non-null  object 
 2   surface                          50470 non-null  object 
 3   draw_size                        50470 non-null  int64  
 4   tourney_level                    50470 non-null  object 
 5   tourney_date                     50470 non-null  object 
 6   match_num                        50470 non-null  int64  
 7   score                            50470 non-null  object 
 8   best_of                          50470 non-null  int64  
 9   round                            50470 non-null  object 
 10  minutes                          50470 non-null  float64
 11  player1                          50470 non-null  object 
 12  player2           

In [6]:
# clean court surface encoded columns
wf.columns = wf.columns.str.replace('surface_', '')
wf = wf.rename(columns = {'Carpet': 'carpet',
                          'Clay': 'clay',
                          'Grass': 'grass',
                          'Hard': 'hard_court'
                         }
              )

In [7]:
# clean tourney level encoded columns
wf.columns = wf.columns.str.replace('tourney_level_', '')
wf = wf.rename(columns = {'A': 'basic_tour',
                          'D': 'davis_cup',
                          'F': 'season_ending_event',
                          'G': 'grand_slam',
                          'M': 'masters_1000'
                         })

wf.columns = wf.columns.str.replace('round_', '')
wf = wf.rename(columns = {'BR': 'bronze_round',
                          'ER': 'early_rounds',
                          'F': 'final',
                          'QF': 'quarter_final',
                          'R128': 'round_of_128',
                          'R16': 'last_16',
                          'R32': 'round_of_32',
                          'R64': 'round_of_64',
                          'RR': 'round_robin',
                          'SF': 'semi_final'
                         })

In [8]:
wf.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,score,best_of,round,minutes,player1,player2,player1_age,player2_age,player1_hand,player2_hand,player1_ht,player2_ht,player1_id,player2_id,player1_ioc,player2_ioc,player1_name,player2_name,player1_rank,player2_rank,player1_rank_points,player2_rank_points,player1_seed,player2_seed,player1_aces,player2_aces,player1_double_faults,player2_double_faults,player1_service_points,player2_service_points,player1_first_serves_in,player2_first_serves_in,player1_first_serve_points_won,player2_first_serve_points_won,player1_second_serve_points_won,player2_second_serve_points_won,player1_service_game_total,player2_service_game_total,player1_break_points_saved,player2_break_points_saved,player1_break_points_faced,player2_break_points_faced,winner,player1_first_serve_%,player2_first_serve_%,player1_first_serve_win_%,player2_first_serve_win_%,player1_break_points_won,player2_break_points_won,player1_seeded,player2_seeded,carpet,clay,grass,hard_court,basic_tour,davis_cup,season_ending_event,grand_slam,masters_1000,player1_hand_L,player1_hand_R,player2_hand_L,player2_hand_R,bronze_round,early_rounds,final,quarter_final,round_of_128,last_16,round_of_32,round_of_64,round_robin,semi_final,h2h_1,h2h_2,player1_wins
0,1999-339,Adelaide,Hard,32,A,1999-01-04,2,7-5 6-3,3,R32,71.0,Jim Courier,Slava Dosedel,28.383299,28.391513,R,R,185.0,183.0,101793,101792,USA,CZE,Jim Courier,Slava Dosedel,77.0,64.0,635.0,723.0,NaN,NaN,3.0,5.0,2.0,0.0,71.0,53.0,41.0,37.0,31.0,29.0,12.0,12.0,11.0,10.0,5.0,1.0,8.0,1.0,Slava Dosedel,0.577465,0.698113,0.756098,0.783784,0.0,3.0,False,False,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,False
1,1999-339,Adelaide,Hard,32,A,1999-01-04,3,6-2 6-1,3,R32,45.0,Andrew Ilie,Jiri Novak,22.713210,23.789185,R,R,180.0,190.0,102776,102562,AUS,CZE,Andrew Ilie,Jiri Novak,59.0,75.0,757.0,669.0,NaN,NaN,2.0,1.0,4.0,2.0,47.0,41.0,22.0,24.0,12.0,19.0,8.0,11.0,8.0,7.0,3.0,0.0,8.0,0.0,Jiri Novak,0.468085,0.585366,0.545455,0.791667,0.0,5.0,False,False,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,False
2,1999-339,Adelaide,Hard,32,A,1999-01-04,4,7-5 6-4,3,R32,86.0,Lleyton Hewitt,Nicolas Kiefer,17.859001,21.500342,R,R,180.0,183.0,103720,103017,AUS,GER,Lleyton Hewitt,Nicolas Kiefer,100.0,35.0,482.0,1007.0,NaN,7.0,4.0,5.0,1.0,3.0,68.0,74.0,50.0,44.0,37.0,30.0,11.0,17.0,11.0,11.0,1.0,8.0,1.0,10.0,Lleyton Hewitt,0.735294,0.594595,0.740000,0.681818,2.0,0.0,False,True,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,True
3,1999-339,Adelaide,Hard,32,A,1999-01-04,5,6-1 6-4,3,R32,61.0,Arnaud Di Pasquale,Jason Stoltenberg,19.895962,28.752909,R,R,185.0,185.0,103324,101727,FRA,AUS,Arnaud Di Pasquale,Jason Stoltenberg,81.0,27.0,613.0,1280.0,NaN,4.0,2.0,7.0,2.0,3.0,52.0,57.0,25.0,32.0,16.0,28.0,13.0,12.0,8.0,9.0,4.0,1.0,7.0,1.0,Jason Stoltenberg,0.480769,0.561404,0.640000,0.875000,0.0,3.0,False,True,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,False
4,1999-339,Adelaide,Hard,32,A,1999-01-04,7,6-3 6-3,3,R32,68.0,Andrei Pavel,Mikael Tillstrom,24.936345,26.833676,R,R,183.0,185.0,102318,102025,ROU,SWE,Andrei Pavel,Mikael Tillstrom,68.0,85.0,710.0,588.0,NaN,NaN,5.0,5.0,2.0,4.0,54.0,79.0,41.0,51.0,28.0,28.0,8.0,12.0,9.0,9.0,3.0,7.0,4.0,11.0,Andrei Pavel,0.759259,0.645570,0.682927,0.549020,4.0,1.0,False,False,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,True


In [9]:
%%time

sf = wf.groupby(['player1', 'player2', 'surface'])['winner']

for key, item in sf:
    print(sf.get_group(key), '\n\n')

31088    Albert Ramos
42455    Albert Ramos
49398    Albert Ramos
Name: winner, dtype: object 


39839    Adrian Mannarino
43696    Adrian Mannarino
50168    Adrian Mannarino
Name: winner, dtype: object 


30056     Alejandro Falla
39281    Adrian Mannarino
Name: winner, dtype: object 


35998      Adrian Mannarino
37386    Alejandro Gonzalez
Name: winner, dtype: object 


31291    Alex Bogomolov Jr
Name: winner, dtype: object 


50169    Alex De Minaur
Name: winner, dtype: object 


45192    Alexander Zverev
Name: winner, dtype: object 


43788    Alexander Zverev
Name: winner, dtype: object 


39001    Adrian Mannarino
Name: winner, dtype: object 


40527    Adrian Mannarino
Name: winner, dtype: object 


49042    Adrian Mannarino
Name: winner, dtype: object 


37903    Andreas Seppi
Name: winner, dtype: object 


42793    Adrian Mannarino
42900       Andreas Seppi
45278    Adrian Mannarino
Name: winner, dtype: object 


24339    Adrian Mannarino
40964       Andreas Seppi
48256      

10105    Albert Costa
Name: winner, dtype: object 


6870    Roger Federer
Name: winner, dtype: object 


12942    Albert Costa
Name: winner, dtype: object 


6781     Roger Federer
10240     Albert Costa
13918    Roger Federer
Name: winner, dtype: object 


1801       Albert Costa
2925    Sargis Sargsian
Name: winner, dtype: object 


11762    Albert Costa
Name: winner, dtype: object 


9734    Albert Costa
Name: winner, dtype: object 


7854    Sebastien Grosjean
Name: winner, dtype: object 


1537    Sebastien Grosjean
6844    Sebastien Grosjean
Name: winner, dtype: object 


4998    Sergi Bruguera
Name: winner, dtype: object 


10669    Albert Costa
Name: winner, dtype: object 


2679    Sjeng Schalken
Name: winner, dtype: object 


5361    Albert Costa
Name: winner, dtype: object 


461    Albert Costa
Name: winner, dtype: object 


1204      Albert Costa
3524      Albert Costa
8081     Stefan Koubek
15302     Albert Costa
Name: winner, dtype: object 


2369    Albert Costa
2646  

Name: winner, dtype: object 


42464    Albert Ramos
Name: winner, dtype: object 


36667    Karen Khachanov
43664    Karen Khachanov
47907    Karen Khachanov
47986    Karen Khachanov
Name: winner, dtype: object 


32768    Kei Nishikori
32846    Kei Nishikori
35279     Albert Ramos
Name: winner, dtype: object 


34108    Kei Nishikori
43307    Kei Nishikori
Name: winner, dtype: object 


48331    Kyle Edmund
Name: winner, dtype: object 


44684    Albert Ramos
49462     Laslo Djere
Name: winner, dtype: object 


29830      Albert Ramos
34872      Albert Ramos
36040    Leonardo Mayer
37255      Albert Ramos
39535    Leonardo Mayer
44325      Albert Ramos
45499    Leonardo Mayer
47738    Leonardo Mayer
Name: winner, dtype: object 


37717    Albert Ramos
Name: winner, dtype: object 


48878    Albert Ramos
Name: winner, dtype: object 


44768    Albert Ramos
45142    Albert Ramos
Name: winner, dtype: object 


39280    Lucas Pouille
49077     Albert Ramos
Name: winner, dtype: object 




Name: winner, dtype: object 


18269    Marc Gicquel
20562    Marc Gicquel
Name: winner, dtype: object 


20467    Nicolas Almagro
Name: winner, dtype: object 


15987    Nicolas Kiefer
Name: winner, dtype: object 


17983    Nicolas Massu
Name: winner, dtype: object 


12929    Paradorn Srichaphan
Name: winner, dtype: object 


18713    Paul Henri Mathieu
Name: winner, dtype: object 


18631    Philipp Kohlschreiber
Name: winner, dtype: object 


17951    Alessio Di Mauro
Name: winner, dtype: object 


16218    Alessio Di Mauro
Name: winner, dtype: object 


19303    Ruben Ramirez Hidalgo
20150         Alessio Di Mauro
Name: winner, dtype: object 


19263    Sergio Roitman
Name: winner, dtype: object 


18856    Simon Greul
Name: winner, dtype: object 


17929    Alessio Di Mauro
28105       Stan Wawrinka
Name: winner, dtype: object 


18128    Tim Henman
Name: winner, dtype: object 


33401    Alexandr Dolgopolov
Name: winner, dtype: object 


13591    Andre Agassi
Name: winner, dtyp

Name: winner, dtype: object 


8331    Nicolas Kiefer
Name: winner, dtype: object 


11718    Alexander Popp
Name: winner, dtype: object 


13376    Alexander Popp
Name: winner, dtype: object 


10989    Alexander Popp
Name: winner, dtype: object 


4610    Alexander Popp
Name: winner, dtype: object 


3511    Patrick Rafter
Name: winner, dtype: object 


13292    Philipp Petzschner
Name: winner, dtype: object 


13935    Potito Starace
Name: winner, dtype: object 


10961    Alexander Popp
Name: winner, dtype: object 


7947    Rainer Schuettler
Name: winner, dtype: object 


16783    Robby Ginepri
Name: winner, dtype: object 


9053    Sargis Sargsian
Name: winner, dtype: object 


16326    Sebastien Grosjean
Name: winner, dtype: object 


17033    Sergiy Stakhovsky
Name: winner, dtype: object 


10713    Stefan Koubek
Name: winner, dtype: object 


8325    Alexander Popp
Name: winner, dtype: object 


8520     Taylor Dent
15919    Taylor Dent
Name: winner, dtype: object 


6413     

...

Name: winner, dtype: object 


20973    Novak Djokovic
Name: winner, dtype: object 


24191    Novak Djokovic
Name: winner, dtype: object 


42543    Novak Djokovic
42749    Novak Djokovic
44991    Novak Djokovic
47395    Novak Djokovic
Name: winner, dtype: object 


49742    Novak Djokovic
Name: winner, dtype: object 


34897           Novak Djokovic
37296           Novak Djokovic
41151           Novak Djokovic
43772    Roberto Bautista Agut
48517    Roberto Bautista Agut
49104    Roberto Bautista Agut
Name: winner, dtype: object 


50024    Novak Djokovic
Name: winner, dtype: object 


32804    Novak Djokovic
Name: winner, dtype: object 


21693    Novak Djokovic
29462    Novak Djokovic
Name: winner, dtype: object 


20726    Novak Djokovic
30612    Novak Djokovic
Name: winner, dtype: object 


23908     Novak Djokovic
24269     Novak Djokovic
26782     Novak Djokovic
27067     Novak Djokovic
27082    Robin Soderling
Name: winner, dtype: object 


17371    Novak Djokovic
Name: winner

Name: winner, dtype: object 


16085    Vincent Spadea
Name: winner, dtype: object 


17612    Paul Goldstein
Name: winner, dtype: object 


7369    Paul Goldstein
Name: winner, dtype: object 


543    Wayne Black
Name: winner, dtype: object 


18493    Paul Goldstein
Name: winner, dtype: object 


791    Xavier Malisse
Name: winner, dtype: object 


2345     Paul Goldstein
4937     Xavier Malisse
17618    Paul Goldstein
Name: winner, dtype: object 


26187    Paul Henri Mathieu
28635             Pere Riba
Name: winner, dtype: object 


8955    Paul Henri Mathieu
Name: winner, dtype: object 


11007       Paul Henri Mathieu
40848    Philipp Kohlschreiber
Name: winner, dtype: object 


26645    Philipp Kohlschreiber
39318    Philipp Kohlschreiber
39444    Philipp Kohlschreiber
Name: winner, dtype: object 


25221    Paul Henri Mathieu
Name: winner, dtype: object 


14212    Radek Stepanek
17090    Radek Stepanek
Name: winner, dtype: object 


18036        Radek Stepanek
21217    Paul He

Name: winner, dtype: object 


16413    Rafael Nadal
17758    Rafael Nadal
Name: winner, dtype: object 


40922    Rafael Nadal
Name: winner, dtype: object 


33159    Rafael Nadal
40166    Rafael Nadal
47309    Rafael Nadal
Name: winner, dtype: object 


24931    Rafael Nadal
27126    Rafael Nadal
39053    Rafael Nadal
Name: winner, dtype: object 


12047    Rafael Nadal
Name: winner, dtype: object 


30170    Rafael Nadal
Name: winner, dtype: object 


21319     Rafael Nadal
28033     Rafael Nadal
32807     Rafael Nadal
35416     Rafael Nadal
35631     Rafael Nadal
40223    Stan Wawrinka
42356     Rafael Nadal
45158     Rafael Nadal
49337     Rafael Nadal
Name: winner, dtype: object 


19927     Rafael Nadal
22170     Rafael Nadal
25339     Rafael Nadal
28834     Rafael Nadal
29253     Rafael Nadal
36618     Rafael Nadal
36799     Rafael Nadal
37052    Stan Wawrinka
41357     Rafael Nadal
41538    Stan Wawrinka
41547     Rafael Nadal
47887     Rafael Nadal
50399     Rafael Nadal
Name

Name: winner, dtype: object 


19332    Stefano Galvani
Name: winner, dtype: object 


17637    Robin Vik
Name: winner, dtype: object 


19081    Xavier Malisse
Name: winner, dtype: object 


46560    Roger Federer
Name: winner, dtype: object 


23047    Roger Federer
Name: winner, dtype: object 


30177    Roger Federer
32663    Roger Federer
Name: winner, dtype: object 


40655    Roger Federer
Name: winner, dtype: object 


38750    Roger Federer
Name: winner, dtype: object 


23331    Roger Federer
Name: winner, dtype: object 


40639    Roger Federer
Name: winner, dtype: object 


20387    Roger Federer
Name: winner, dtype: object 


38246    Roger Federer
Name: winner, dtype: object 


31453    Roger Federer
Name: winner, dtype: object 


7282    Roger Federer
Name: winner, dtype: object 


5727     Roger Federer
10590    Roger Federer
Name: winner, dtype: object 


10820    Roger Federer
Name: winner, dtype: object 


11370    Roger Federer
Name: winner, dtype: object 


5316   

48663    Stefanos Tsitsipas
Name: winner, dtype: object 


45010    Steve Johnson
Name: winner, dtype: object 


44037    Steve Johnson
Name: winner, dtype: object 


43737    Taylor Fritz
Name: winner, dtype: object 


42478    Stephane Robert
Name: winner, dtype: object 


40760    Thomaz Bellucci
Name: winner, dtype: object 


44595    Thomaz Bellucci
Name: winner, dtype: object 


28224    Tobias Kamke
Name: winner, dtype: object 


30703    Stephane Robert
Name: winner, dtype: object 


37498    Tomas Berdych
Name: winner, dtype: object 


37387    Victor Hanescu
Name: winner, dtype: object 


39579    Viktor Troicki
Name: winner, dtype: object 


29601    Xavier Malisse
Name: winner, dtype: object 


32328    Steve Darcis
Name: winner, dtype: object 


39431    Steve Darcis
Name: winner, dtype: object 


24630    Taylor Dent
Name: winner, dtype: object 


43563    Steve Darcis
Name: winner, dtype: object 


33687     Steve Darcis
45207    Tomas Berdych
Name: winner, dtype: object

In [16]:
# set empty dataframe to fill
surface_wins = pd.DataFrame()
target_cols = ['winner', 'surface']

for groups, winner in wf.groupby(['player1', 'player2', 'surface'])['winner']:
    num_matches = len(groups)
    print(f'groups: {groups}')
    print(f'winner: {winner}')                                            # print winner
    print(f'Length of winner: {num_matches}')                                       # print num_matches
    s1 = 0
    s2 = 0
    temp_df = pd.DataFrame()
    for i in range(num_matches):
        print(f'winner {winner}')                                        # print winner
        if winner.iloc[1] == groups[0]:
            s1 = s1 + 1
            temp_df = temp_df.append(pd.DataFrame([[groups[0], groups[1], groups[2], winner, s1, s2]],
                                                  columns = ['player1',
                                                             'player2',
                                                             'surface',
                                                             'winner',
                                                             's1',
                                                             's2'
                                                            ],
                                                  index = [winner.index[i]]))
            
        elif winner.iloc[1] == groups[1]:
            s2 = s2 + 1
            temp_df = temp_df.append(pd.DataFrame([[groups[0], groups[1], groups[2], winner, s1, s2]],
                                                  columns = ['player1',
                                                             'player2',
                                                             'surface',
                                                             'winner',
                                                             's1',
                                                             's2'
                                                            ],
                                                  index = [winner.index[i]]))
            
            
    print(temp_df)
    temp_df[['s1', 's2']] = temp_df[['s1', 's2']].shift(1).fillna(0)
    surface_wins = surface_wins.append(temp_df)

surface_wins

groups: ('Adrian Mannarino', 'Albert Ramos', 'Clay')
winner: 31088    Albert Ramos
42455    Albert Ramos
49398    Albert Ramos
Name: winner, dtype: object
Length of winner: 3
winner 31088    Albert Ramos
42455    Albert Ramos
49398    Albert Ramos
Name: winner, dtype: object
winner 31088    Albert Ramos
42455    Albert Ramos
49398    Albert Ramos
Name: winner, dtype: object
winner 31088    Albert Ramos
42455    Albert Ramos
49398    Albert Ramos
Name: winner, dtype: object
                player1       player2 surface  \
31088  Adrian Mannarino  Albert Ramos    Clay   
42455  Adrian Mannarino  Albert Ramos    Clay   
49398  Adrian Mannarino  Albert Ramos    Clay   

                                                  winner  s1  s2  
31088  31088    Albert Ramos
42455    Albert Ramos
49...   0   1  
42455  31088    Albert Ramos
42455    Albert Ramos
49...   0   2  
49398  31088    Albert Ramos
42455    Albert Ramos
49...   0   3  
groups: ('Adrian Mannarino', 'Albert Ramos', 'Hard')
winn

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
# create a empty dataframe
h2h_df = pd.DataFrame()

# loop through each pair of players
for players, winner in df.groupby(['player_1', 'player_2', 'surface'])['winner']:
    num_matches = len(winner)
    p1 = 0
    p2 = 0
    temp_df = pd.DataFrame()
    for i in range(0, num_matches):
        if winner.iloc[i] == players[0]:
            p1 = p1 + 1
            temp_df = temp_df.append(pd.DataFrame([[players[0], players[1], p1, p2]], columns = ['player_1', 'player_2', 'h2h_1', 'h2h_2'], index = [winner.index[i]]))
        else:
            p2 = p2 + 1 
            temp_df = temp_df.append(pd.DataFrame([[players[0], players[1], p1, p2]], columns = ['player_1', 'player_2', 'h2h_1', 'h2h_2'], index = [winner.index[i]]))


    # shift the stats by 1 row so we're not cheating on h2h record a match early
    temp_df[['h2h_1', 'h2h_2']] = temp_df[['h2h_1', 'h2h_2']].shift(1).fillna(0)
    h2h_df = h2h_df.append(temp_df)

# join h2h info with df
df = df.join(h2h_df[['h2h_1', 'h2h_2']])

# marginally reduce noise
df.h2h_1 = df.h2h_1.astype(int)
df.h2h_2 = df.h2h_2.astype(int)